# 第10章
---
# 时间序列

不管在哪个领域中，时间序列（time series）数据都是一种重要的结构化数据形式。在多个时间点观察或测量到的任何事物都可以形成一段时间序列。时间序列有固定频率的，也可以是不定期的。数据的意义取决于具体的应用场景，主要有以下几种：

- 时间戳（timestamp），特定的时刻
- 固定时期（period），如2007年1月或2010年全年
- 时间间隔（interval），由起始和结束时间戳表示。时期（period）可以被看做间隔（interval）的特例
- 实验或过程时间，每个时间点都是相对于特定起始时间的一个度量。例如，从放入烤箱时起，每秒钟饼干的直径。

本章主要讲解前三种时间序列，timestamp、period、interval。许多技术都可以用于处理实验型时间序列，其索引可能是一个整数或浮点数（表示从实验开始算起已经过去的时间）。最简单也最常见的时间序列都是用时间戳进行索引的。

pandas提供的时间序列处理工具和数据算法：轻松的切片/切块、聚合、对定期/不定期的时间序列进行重采样等。对金融和经济数据尤为游泳，当然也可以分析任何带时间序列的数据集，如服务器日志数据。

## 日期和时间数据类型及工具

Python标准库包含用于日期（date）和时间（time）数据的数据类型，且有日历方面的功能。主要用到**datetime、time、以及calendar模块**。datetime.datetime是用的最多的数据类型：

In [1]:
from datetime import datetime

now = datetime.now()
now

datetime.datetime(2016, 7, 12, 9, 36, 36, 144000)

In [2]:
now.year, now.month, now.day

(2016, 7, 12)

datetime以毫秒形式存储日期和时间。datetime.timedelta表示两个datetime对象之间的时间差：

In [3]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta

datetime.timedelta(926, 56700)

In [4]:
delta.days

926

In [5]:
delta.seconds

56700

可以给datetime对象加上（或减去）一个或多个timedelta，这样会产生一个新对象：

In [6]:
from datetime import timedelta

start = datetime(2016, 1, 7)
start + timedelta(10)

datetime.datetime(2016, 1, 17, 0, 0)

In [7]:
start - 2 * timedelta(11, 11)

datetime.datetime(2015, 12, 15, 23, 59, 38)

Python datetime的数据类型

类型 | 说明
- | -
date | 以公历形式存储日历日期（年、月、日）
time | 将时间存储为时、分、秒、毫秒
datetime | 存储日期和时间
timedelta | 表示两个datetime值之间的差（日、秒、毫秒）

### 字符串和datetime的相互转换
利用str或strftime方法（传入一个格式化字符串），datetime对象和pandas的Timestamp对象可以被格式化为字符串：

In [8]:
stamp = datetime(2011, 1, 3)

str(stamp)

'2011-01-03 00:00:00'

In [9]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

In [10]:
value = '2016-07-05'

datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2016, 7, 5, 0, 0)

In [11]:
datestrs = ['7/11/2016', '7/15/2016']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2016, 7, 11, 0, 0), datetime.datetime(2016, 7, 15, 0, 0)]

datetime.strptime是通过已知格式进行日期解析的最佳方式。但是每次都要编写格式定义是很麻烦的事情，尤其一些常见的格式。这样可以用deteutil这个三方包中的parser.parse方法：

In [12]:
from dateutil.parser import parse

parse('2011/07/19')

datetime.datetime(2011, 7, 19, 0, 0)

In [13]:
# dateutil可以寄解析几乎所有人能够理解的日期形式：（中文好像不行）
parse('Jan 31, 1997 10:22 PM')

datetime.datetime(1997, 1, 31, 22, 22)

In [14]:
# 国际通用格式中，经常有日在前的，传入dayfirst=True即可：
parse('6/12/2016', dayfirst=True)

datetime.datetime(2016, 12, 6, 0, 0)

pandas通常是用于处理成组日期的，不管是DataFrame的轴索引还是列。to_datetime的方法可以解析多种不同的日期表示形式。对标准日期如ISO8601的解析非常快。

In [15]:
datestrs

['7/11/2016', '7/15/2016']

In [16]:
import pandas as pd
pd.to_datetime(datestrs)

DatetimeIndex(['2016-07-11', '2016-07-15'], dtype='datetime64[ns]', freq=None)

In [17]:
# 处理缺失值（None、空字符串等）:
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2016-07-11', '2016-07-15', 'NaT'], dtype='datetime64[ns]', freq=None)

In [18]:
idx[2] # not a time pandas中时间戳数据的NA值

NaT

In [19]:
pd.isnull(idx)

array([False, False,  True], dtype=bool)

**警告：**dateutil.parser会把一些不是日期的字符串认作是日期（比如“42”会被解析为2042年的今天）。

datetime格式定义（兼容ISO C89）

代码 | 说明
- | -
%Y | 4位数的年
%y | 2位数的年
%m | 2位数的月[01,12]
%d | 2位数的日[01,31]
%H | 时（24小时制）[00,23]
%I | 时（12小时制）[01,12]
%M | 2位数的分[00,59]
%S | 秒[00,61]（秒60和秒61用于闰秒）
%w | 用整数表示的星期几[0(星期天)，6]
%U | 每年的第几周[00,53]。星期天被任务是每周的第一天，每年第一个星期天之前的那几天被任务是“第0周”
%W | 每年的第几周[00,53]。星期一被任务是每周的第一天，每年第一个星期一之前的那几天被任务是“第0周”
%z | 以+HHMM或-HHMM表示的UTC时区偏移量，如果时间对象为naive，则返回空字符串
%F | %Y-%m-%d简写形式，例如2016-07-11
%D | %m/%d/%y简写形式，例如11/07/12

datetime对象还有一些特定于当前环境（位于不同国家或使用不同语言的系统）的格式化选项。例如，德语或法语系统所用的鱼粉简写就与英语系统所用的不同。

特定于当前环境的日期格式

代码 | 说明
- | -
%a | 星期几的简写
%A | 星期几的全称
%b | 月份的简写
%B | 月份的全称
%c | 完整的日期和时间，例如“Tue 01 May 2012 04:20:57 PM”
%p | 不同环境中的AM或PM
%x | 适合于当前环境的日期格式，例如，在美国，“May 1,2012”会产生“05/01/2012”
%X | 适合于当前环境的时间格式，例如“04:24:12 PM”

## 时间序列基础
pandas最基本的时间序列类型就是以时间戳（通常以Python字符串或datetime对象表示）为索引的Series：

In [21]:
from datetime import datetime
from pandas import Series, DataFrame
import numpy as np

dates = [datetime(2016, 7, 12), datetime(2016,7, 15), datetime(2016,7, 18),
         datetime(2016,7, 19), datetime(2016,7, 20), datetime(2016,7, 22)]
ts = Series(np.random.randn(6), index=dates)

ts

2016-07-12   -0.045113
2016-07-15    0.649828
2016-07-18   -1.454268
2016-07-19    0.390478
2016-07-20   -1.314967
2016-07-22    1.232518
dtype: float64

这些datetime对象实际上是被放在一个DatetimeIndex中的。现在，变量ts成为了一个TimeSeries了：

In [22]:
type(ts)

pandas.core.series.Series

In [23]:
ts.index

DatetimeIndex(['2016-07-12', '2016-07-15', '2016-07-18', '2016-07-19',
               '2016-07-20', '2016-07-22'],
              dtype='datetime64[ns]', freq=None)

In [25]:
# 跟其他Series一样，不同索引的时间序列之间的算数运算会自动按日期对齐：
ts + ts[::2]

2016-07-12   -0.090225
2016-07-15         NaN
2016-07-18   -2.908536
2016-07-19         NaN
2016-07-20   -2.629934
2016-07-22         NaN
dtype: float64

In [26]:
# pandas用Numpy的datetime64数据类型以纳秒形式存储时间戳：
ts.index.dtype

dtype('<M8[ns]')

In [27]:
# DatetimeIndex中的各个标量值是pandas的Timestamp对象：
stamp = ts.index[0]

In [28]:
stamp

Timestamp('2016-07-12 00:00:00')

只要有需要，Timestamp可以随时自动转换为datetime对象。此外，它还可以储存频率信息，且知道如何执行时区转换以及其他操作。

### 索引、选取、子集构造
由于TimeSeries是Series的一个子类，所以在索引以及数据选取方面它们的行为是一样的：

In [29]:
stamp = ts.index[2]
ts[stamp]

-1.454267891179267

In [30]:
# 传入一个可以被解释为日期的字符串：
ts['7/18/2016']

-1.454267891179267

In [31]:
ts['20160718']

-1.454267891179267

In [33]:
# 对于较长的时间序列，只需传入年或年月即可轻松选取数据的切片：
longer_ts = Series(np.random.randn(1000), index=pd.date_range('1/1/2016', periods=1000))
longer_ts

2016-01-01   -0.675725
2016-01-02   -0.141727
2016-01-03    0.138097
2016-01-04   -0.506381
2016-01-05    0.426840
2016-01-06    0.740131
2016-01-07   -0.261179
2016-01-08   -1.785051
2016-01-09   -0.271962
2016-01-10   -0.563065
2016-01-11    1.194537
2016-01-12    0.775137
2016-01-13   -1.446761
2016-01-14    1.021178
2016-01-15    0.829213
2016-01-16   -0.781522
2016-01-17   -0.226336
2016-01-18    2.037136
2016-01-19   -0.814444
2016-01-20   -1.355549
2016-01-21    0.625121
2016-01-22   -0.339175
2016-01-23    1.205029
2016-01-24    0.273884
2016-01-25    0.435693
2016-01-26   -0.836612
2016-01-27   -0.254710
2016-01-28   -1.677453
2016-01-29    0.253607
2016-01-30    1.120106
                ...   
2018-08-28   -1.243718
2018-08-29   -0.791021
2018-08-30    0.707489
2018-08-31   -0.030824
2018-09-01   -0.551495
2018-09-02   -1.005471
2018-09-03    0.370616
2018-09-04    0.336487
2018-09-05    1.667086
2018-09-06    0.700433
2018-09-07   -0.671798
2018-09-08    0.586417
2018-09-09 

In [34]:
longer_ts['2017']

2017-01-01   -0.724404
2017-01-02   -0.151567
2017-01-03   -1.255442
2017-01-04    1.346502
2017-01-05    0.789671
2017-01-06   -0.902544
2017-01-07    0.942987
2017-01-08   -0.918491
2017-01-09    1.165144
2017-01-10    0.545001
2017-01-11    0.120349
2017-01-12   -0.848383
2017-01-13   -0.943522
2017-01-14    0.020188
2017-01-15   -0.040236
2017-01-16   -0.826514
2017-01-17    0.518047
2017-01-18   -1.967786
2017-01-19    0.839364
2017-01-20    2.007676
2017-01-21    1.126215
2017-01-22    0.826396
2017-01-23   -2.350563
2017-01-24   -0.754830
2017-01-25    0.992942
2017-01-26    0.501965
2017-01-27   -0.055375
2017-01-28    0.786970
2017-01-29   -0.751034
2017-01-30    0.547839
                ...   
2017-12-02   -1.873843
2017-12-03    0.315243
2017-12-04    1.877256
2017-12-05   -0.115512
2017-12-06   -1.475026
2017-12-07   -0.828618
2017-12-08   -0.841858
2017-12-09   -0.967887
2017-12-10   -1.381007
2017-12-11   -0.272476
2017-12-12    1.037558
2017-12-13   -1.173636
2017-12-14 

In [35]:
longer_ts['2018-03']

2018-03-01    0.589136
2018-03-02    0.639450
2018-03-03   -0.714187
2018-03-04    0.822019
2018-03-05   -0.423471
2018-03-06    1.781130
2018-03-07   -0.276728
2018-03-08   -1.316685
2018-03-09    0.909355
2018-03-10   -0.139407
2018-03-11   -1.100306
2018-03-12    0.465655
2018-03-13   -2.122935
2018-03-14   -0.405505
2018-03-15   -1.178399
2018-03-16   -1.238570
2018-03-17    0.857985
2018-03-18   -0.012980
2018-03-19   -0.190398
2018-03-20   -0.522460
2018-03-21    0.588029
2018-03-22    1.151910
2018-03-23    1.102607
2018-03-24    0.646131
2018-03-25    0.995800
2018-03-26   -0.174346
2018-03-27   -0.848613
2018-03-28   -1.220765
2018-03-29    2.609912
2018-03-30   -0.725234
2018-03-31   -0.743960
Freq: D, dtype: float64

In [37]:
# 通过日期进行切片的方式只对规则Series有效：
ts[datetime(2016, 7, 15):]

2016-07-15    0.649828
2016-07-18   -1.454268
2016-07-19    0.390478
2016-07-20   -1.314967
2016-07-22    1.232518
dtype: float64

In [38]:
# 大部分时间序列数据都是按照时间先后排序的，可以用不存在与该时间序列中的时间戳对其进行切片（范围查询）：
ts

2016-07-12   -0.045113
2016-07-15    0.649828
2016-07-18   -1.454268
2016-07-19    0.390478
2016-07-20   -1.314967
2016-07-22    1.232518
dtype: float64

In [39]:
ts['7/13/2016':'7/21/2016']

2016-07-15    0.649828
2016-07-18   -1.454268
2016-07-19    0.390478
2016-07-20   -1.314967
dtype: float64

根之前一样，这里可以传入字符串日期、datetime或Timestamp。注意，这样切片所产生的是源时间序列的视图，根NumPy数组的切片运算是一样的。此外，还有一个等价的实例方法也可以截取两个日期之间TimeSeries：

In [40]:
ts.truncate(after='7/16/2016')

2016-07-12   -0.045113
2016-07-15    0.649828
dtype: float64

In [41]:
# 上面这些操作对DataFrame也有效。例如，对DataFrame的行进行索引：
dates = pd.date_range('1/1/2016', periods=100, freq='W-WED')
long_df = DataFrame(np.random.randn(100, 4),
                   index=dates,
                   columns=['Colorado', 'Texas', 'New York', 'Ohio'])

long_df.ix['5-2016']

,Colorado,Texas,New York,Ohio
2016-05-04,-0.382943,0.236260,0.090642,-1.213911
2016-05-11,0.727608,0.314077,0.916490,0.941340
2016-05-18,-2.011003,-1.397905,0.297171,-0.696944
2016-05-25,-0.151385,0.099110,1.031737,0.770096


### 带有重复索引的时间序列
在某些场景中国，可能会存在多个观测数据落在同一个时间点上的情况。下面就是一个例子：

In [43]:
dates = pd.DatetimeIndex(['1/1/2016', '1/2/2016', '1/2/2016', '1/2/2016', '1/3/2016'])
dup_ts = Series(np.arange(5), index=dates)

dup_ts

2016-01-01    0
2016-01-02    1
2016-01-02    2
2016-01-02    3
2016-01-03    4
dtype: int32

In [44]:
# 通过检查索引的is_unique属性，我们就可以知道它是不是唯一的：
dup_ts.index.is_unique

False

In [45]:
# 索引时可能会产生标量值或切片，具体看时间点是否重复：
dup_ts['2016/01/01']

0

In [46]:
dup_ts['2016/01/02']

2016-01-02    1
2016-01-02    2
2016-01-02    3
dtype: int32

In [47]:
# 聚合非唯一时间戳，可以使用groupby并传入level=0（索引的唯一一层！）：
grouped = dup_ts.groupby(level=0)
grouped.mean()

2016-01-01    0
2016-01-02    2
2016-01-03    4
dtype: int32

In [48]:
grouped.count()

2016-01-01    1
2016-01-02    3
2016-01-03    1
dtype: int64

##  日期的范围、频率以及移动
pandas中的时间序列一般被任务是不规则的，它没有固定的频率，对大部分应用而言，这是无所谓的，但是它常常需要以。某种相对规定的频率进行分析，比如每日、每月、每15分钟等（这样自然会在时间序列中引入缺失值）。还好，*pandas有套标准时间序列频率以及用于重采样、频率推断、生成固定频率日期范围的工具*。例如，将之前时间序列转换为一个具有固定频率（每日）的时间序列，只需要调用resample即可：

In [49]:
ts

2016-07-12   -0.045113
2016-07-15    0.649828
2016-07-18   -1.454268
2016-07-19    0.390478
2016-07-20   -1.314967
2016-07-22    1.232518
dtype: float64

In [101]:
ts.resample('D').mean()

2016-01-31    0.204476
2016-02-29    0.842104
2016-03-31    0.320751
Freq: M, dtype: float64

频率的转换（或重采样）是一个比较大的主题，稍后将专门用一节讨论。

### 生成日期范围
pandas.date_range可用于生成指定长度的DatetimeIndex：

In [58]:
index = pd.date_range('4/1/2016', '6/1/2016')
index

DatetimeIndex(['2016-04-01', '2016-04-02', '2016-04-03', '2016-04-04',
               '2016-04-05', '2016-04-06', '2016-04-07', '2016-04-08',
               '2016-04-09', '2016-04-10', '2016-04-11', '2016-04-12',
               '2016-04-13', '2016-04-14', '2016-04-15', '2016-04-16',
               '2016-04-17', '2016-04-18', '2016-04-19', '2016-04-20',
               '2016-04-21', '2016-04-22', '2016-04-23', '2016-04-24',
               '2016-04-25', '2016-04-26', '2016-04-27', '2016-04-28',
               '2016-04-29', '2016-04-30', '2016-05-01', '2016-05-02',
               '2016-05-03', '2016-05-04', '2016-05-05', '2016-05-06',
               '2016-05-07', '2016-05-08', '2016-05-09', '2016-05-10',
               '2016-05-11', '2016-05-12', '2016-05-13', '2016-05-14',
               '2016-05-15', '2016-05-16', '2016-05-17', '2016-05-18',
               '2016-05-19', '2016-05-20', '2016-05-21', '2016-05-22',
               '2016-05-23', '2016-05-24', '2016-05-25', '2016-05-26',
      

*默认情况下，date_range会产生按天计算的时间点*，如果只传入起始或结束日期，那就还得传入一个表示一段时间的数字：

In [59]:
pd.date_range(start='4/1/2016', periods=20)

DatetimeIndex(['2016-04-01', '2016-04-02', '2016-04-03', '2016-04-04',
               '2016-04-05', '2016-04-06', '2016-04-07', '2016-04-08',
               '2016-04-09', '2016-04-10', '2016-04-11', '2016-04-12',
               '2016-04-13', '2016-04-14', '2016-04-15', '2016-04-16',
               '2016-04-17', '2016-04-18', '2016-04-19', '2016-04-20'],
              dtype='datetime64[ns]', freq='D')

In [60]:
pd.date_range(end='6/1/2016', periods=20)

DatetimeIndex(['2016-05-13', '2016-05-14', '2016-05-15', '2016-05-16',
               '2016-05-17', '2016-05-18', '2016-05-19', '2016-05-20',
               '2016-05-21', '2016-05-22', '2016-05-23', '2016-05-24',
               '2016-05-25', '2016-05-26', '2016-05-27', '2016-05-28',
               '2016-05-29', '2016-05-30', '2016-05-31', '2016-06-01'],
              dtype='datetime64[ns]', freq='D')

起始和结束日期定义了日期索引的严格边界。例如，如果你想要生成一个由每月最后一个工作日组成的日期索引、可以传入‘BM’频率（business end of month），这样就只会包含时间间隔内（或刚好在边界上的）符合频率要求的日期：

In [65]:
pd.date_range('1/1/2000','12/31/2000', freq='BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30', '2000-12-29'],
              dtype='datetime64[ns]', freq='BM')

In [66]:
# date_range默认会保留起始和结束时间戳的时间信息：
pd.date_range('7/12/2016 12:01:40', periods=5)

DatetimeIndex(['2016-07-12 12:01:40', '2016-07-13 12:01:40',
               '2016-07-14 12:01:40', '2016-07-15 12:01:40',
               '2016-07-16 12:01:40'],
              dtype='datetime64[ns]', freq='D')

In [68]:
# 产生一组被规范化（normalize）到午夜的时间戳。normalize选项即可实现该功能：
pd.date_range('7/12/2016 12:51:42', periods=5, normalize=True)

DatetimeIndex(['2016-07-12', '2016-07-13', '2016-07-14', '2016-07-15',
               '2016-07-16'],
              dtype='datetime64[ns]', freq='D')

### 频率和日期偏移量
pandas中的频率是由一个基础频率（base frequency）和一个乘数组成的。基础频率通常以一个字符串别名表示，比如“M”表示每月，“H”表示每小时。

对于每个基础平率，都有一个被称为**日期偏移量（date offset）**的对象与之对应，例如，按小时计算的频率可以用**Hour类**表示：

In [69]:
from pandas.tseries.offsets import Hour, Minute

hour = Hour()
hour

<Hour>

In [70]:
# 传入一个整数即可定义偏移量的倍数
four_hours = Hour(4)
four_hours

<4 * Hours>

一般，无需显式创建这样的对象，只需使用诸如“H”或“4H”这样的字符串别名即可。在基础频率前面放上一个整数即可创建倍数：

In [72]:
pd.date_range('1/1/2016', '1/3/2016 23:59', freq='4H')

DatetimeIndex(['2016-01-01 00:00:00', '2016-01-01 04:00:00',
               '2016-01-01 08:00:00', '2016-01-01 12:00:00',
               '2016-01-01 16:00:00', '2016-01-01 20:00:00',
               '2016-01-02 00:00:00', '2016-01-02 04:00:00',
               '2016-01-02 08:00:00', '2016-01-02 12:00:00',
               '2016-01-02 16:00:00', '2016-01-02 20:00:00',
               '2016-01-03 00:00:00', '2016-01-03 04:00:00',
               '2016-01-03 08:00:00', '2016-01-03 12:00:00',
               '2016-01-03 16:00:00', '2016-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [73]:
pd.date_range('1/1/2016', '1/3/2016', freq='4H')

DatetimeIndex(['2016-01-01 00:00:00', '2016-01-01 04:00:00',
               '2016-01-01 08:00:00', '2016-01-01 12:00:00',
               '2016-01-01 16:00:00', '2016-01-01 20:00:00',
               '2016-01-02 00:00:00', '2016-01-02 04:00:00',
               '2016-01-02 08:00:00', '2016-01-02 12:00:00',
               '2016-01-02 16:00:00', '2016-01-02 20:00:00',
               '2016-01-03 00:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [74]:
# 大部分偏移量对象都可通过加法链接：
Hour(2) + Minute(28)

<148 * Minutes>

In [77]:
# 2h30min可以高效地解析为等效的表达式：
pd.date_range('1/1/2016', periods=10, freq='1h29min11s')

DatetimeIndex(['2016-01-01 00:00:00', '2016-01-01 01:29:11',
               '2016-01-01 02:58:22', '2016-01-01 04:27:33',
               '2016-01-01 05:56:44', '2016-01-01 07:25:55',
               '2016-01-01 08:55:06', '2016-01-01 10:24:17',
               '2016-01-01 11:53:28', '2016-01-01 13:22:39'],
              dtype='datetime64[ns]', freq='5351S')

有些频率的时间点间隔不均匀，如M（日历月末）和BM（每月最后一个工作日）就取决于每月的天数，对于后者，还要考虑月末是不是周末。将这些成为锚点偏移量（anchored offset）。

pandas中的频率代码和日期偏移量类

别名 | 偏移量类型 | 说明
- | - | -
D | Day | 每日历日
B | BusinessDay | 每工作日
H | Hour | 每小时
T或min | Mniute | 每分
S | Second | 每秒
L或ms | Milli | 每毫秒（即每千分之一秒）
U | Micro | 每微妙（百万分之一秒）
M | MonthEnd | 每月最后一个日历日
BM | BusinessMonthEnd | 每月最后一个工作日
MS | MonthBegin | 每月第一个日历日
BMS | BusinessMonthBegin | 每月第一个工作日

> **详见onenote**

### WOM日期
week of month 非常实用的频率类，它以WOM开头。例如“每月第三个星期五”之类的日期：

In [78]:
rng = pd.date_range('1/1/2016', '9/1/2016', freq='WOM-3FRI')

In [79]:
list(rng)

[Timestamp('2016-01-15 00:00:00', offset='WOM-3FRI'),
 Timestamp('2016-02-19 00:00:00', offset='WOM-3FRI'),
 Timestamp('2016-03-18 00:00:00', offset='WOM-3FRI'),
 Timestamp('2016-04-15 00:00:00', offset='WOM-3FRI'),
 Timestamp('2016-05-20 00:00:00', offset='WOM-3FRI'),
 Timestamp('2016-06-17 00:00:00', offset='WOM-3FRI'),
 Timestamp('2016-07-15 00:00:00', offset='WOM-3FRI'),
 Timestamp('2016-08-19 00:00:00', offset='WOM-3FRI')]

### 移动（超前和之后）数据
移动（shifting）指的是沿着时间轴将数据前移或后移。Series、DF都有一个shift方法用于执行单纯的前移或后移操作，保持索引不变：

In [80]:
ts = Series(np.random.randn(4),
           index=pd.date_range('1/1/2000', periods=4, freq='M'))
ts

2000-01-31   -0.878604
2000-02-29   -0.428289
2000-03-31    2.638179
2000-04-30   -0.755479
Freq: M, dtype: float64

In [81]:
ts.shift(2)

2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -0.878604
2000-04-30   -0.428289
Freq: M, dtype: float64

In [82]:
ts.shift(-2)

2000-01-31    2.638179
2000-02-29   -0.755479
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

shift通常用于计算一个时间序列或多个时间序列中的*百分比变化*：
```
ts / ts.shift(1) -1
```

单纯的位移操作不会修改索引，所以**部分数据会被丢弃**。因此，如果频率已知，则可以传递给shift以实现时间戳位移而不是对数据简单的位移，这样可以避免数据丢失

In [84]:
ts.shift(2, freq='M')

2000-03-31   -0.878604
2000-04-30   -0.428289
2000-05-31    2.638179
2000-06-30   -0.755479
Freq: M, dtype: float64

In [85]:
ts.shift(3, freq='D')

2000-02-03   -0.878604
2000-03-03   -0.428289
2000-04-03    2.638179
2000-05-03   -0.755479
dtype: float64

In [86]:
ts.shift(1, freq='3D')

2000-02-03   -0.878604
2000-03-03   -0.428289
2000-04-03    2.638179
2000-05-03   -0.755479
dtype: float64

In [87]:
ts.shift(1, freq='15T')

2000-01-31 00:15:00   -0.878604
2000-02-29 00:15:00   -0.428289
2000-03-31 00:15:00    2.638179
2000-04-30 00:15:00   -0.755479
Freq: M, dtype: float64

### 通过偏移量对日期进行位移
pandas的日期偏移量还可以用在datetime或Timestamp对象上：

In [89]:
from pandas.tseries.offsets import Day, MonthEnd

now = datetime(2016, 7, 17)
now + 3 * Day()

Timestamp('2016-07-20 00:00:00')

In [90]:
# 如果加锚点偏移量，第一次增量会将原日期向前滚动到符合频率的下一个日期：
now + MonthEnd()

Timestamp('2016-07-31 00:00:00')

In [91]:
now + MonthEnd(2)

Timestamp('2016-08-31 00:00:00')

In [92]:
# 通过锚点偏移量的rollforward和rollback的方法，可显式地将日期向前或向后滚动：
offset = MonthEnd()

offset.rollforward(now)

Timestamp('2016-07-31 00:00:00')

In [93]:
offset.rollback(now)

Timestamp('2016-06-30 00:00:00')

In [95]:
# 结合groupby使用这两个滚动方法：
ts = Series(np.random.randn(20),
           index=pd.date_range('1/15/2016', periods=20, freq='4d'))
ts

2016-01-15   -0.817361
2016-01-19    0.483461
2016-01-23   -0.060212
2016-01-27    2.061389
2016-01-31   -0.644897
2016-02-04    0.605940
2016-02-08    0.195191
2016-02-12    2.468590
2016-02-16    1.668588
2016-02-20    0.060502
2016-02-24   -0.220001
2016-02-28    1.115916
2016-03-03    0.492422
2016-03-07    2.061893
2016-03-11   -0.470021
2016-03-15   -1.695445
2016-03-19    1.490593
2016-03-23    0.158184
2016-03-27    1.729760
2016-03-31   -1.201381
Freq: 4D, dtype: float64

In [98]:
ts.groupby(offset.rollforward).sum()

2016-01-31    1.022379
2016-02-29    5.894726
2016-03-31    2.566006
dtype: float64

In [100]:
# resample可以更快速的实现上述的功能
ts.resample('M').sum()

2016-01-31    1.022379
2016-02-29    5.894726
2016-03-31    2.566006
Freq: M, dtype: float64

## 时区处理
夏令时（DST）转变，许多人选择以协调世界时间来处理时间序列。时区是以UTC偏移量的形式来表示的，例如，夏令时期间，纽约比UTC慢4小时，而在全年其他时间则比UTC慢5小时。

在Python中，时区信息来自[第三方库pytz](http://pytz.sourceforge.net/)，使Python可以使用Olson数据库。pandas包含了pytz的功能，只要记得时区的名称即可，时区名可以在文档中找到，也可以交互的方式查看：

In [118]:
import pytz

pytz.common_timezones[:]

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Tripoli',
 'Africa/Tunis',
 'Africa/Wi

In [103]:
# 获取时区对象：
tz = pytz.timezone('US/Eastern')

In [104]:
tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

### 本地化和转换
默认情况下，pandas中的时间序列是单纯的（naive）时区。

In [106]:
rng = pd.date_range('3/9/2012 9:30', periods=6, freq='D')
ts = Series(np.random.randn(len(rng)), index=rng)

In [107]:
print(ts.index.tz)

None


In [108]:
# 生成时加上时区集
pd.date_range('3/9/2012 9:30', periods=6, freq='D', tz='UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [109]:
# 单纯到本地化的转换是通过tz_localize方法处理的：
ts_utc = ts.tz_localize('UTC')
ts_utc

2012-03-09 09:30:00+00:00   -0.637031
2012-03-10 09:30:00+00:00    0.721287
2012-03-11 09:30:00+00:00   -0.505112
2012-03-12 09:30:00+00:00    0.550210
2012-03-13 09:30:00+00:00    0.912870
2012-03-14 09:30:00+00:00   -2.947034
Freq: D, dtype: float64

In [112]:
ts_utc.index

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [113]:
# 被本地化到特定时区后，可以用tz_convert转换到别的时区：
ts_utc.tz_convert('Asia/Shanghai')

2012-03-09 17:30:00+08:00   -0.637031
2012-03-10 17:30:00+08:00    0.721287
2012-03-11 17:30:00+08:00   -0.505112
2012-03-12 17:30:00+08:00    0.550210
2012-03-13 17:30:00+08:00    0.912870
2012-03-14 17:30:00+08:00   -2.947034
Freq: D, dtype: float64

In [119]:
# tz_localize和tz_convert也是DatetimeIndex的实例方法：
ts.index.tz_localize('Asia/Shanghai')

DatetimeIndex(['2012-03-09 09:30:00+08:00', '2012-03-10 09:30:00+08:00',
               '2012-03-11 09:30:00+08:00', '2012-03-12 09:30:00+08:00',
               '2012-03-13 09:30:00+08:00', '2012-03-14 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq='D')

### 操作时区意识型Timestamp对象
跟时间序列和日期范围差不多，Timestamp对象也能被从单纯型（naive）本地化为时区意识型（time zone-aware），并从一个时区转换到另一个时区：

In [120]:
stamp = pd.Timestamp('2011-03-12 04:00')
stamp_utc = stamp.tz_localize('utc')
stamp_utc.tz_convert('US/Eastern')

Timestamp('2011-03-11 23:00:00-0500', tz='US/Eastern')

In [122]:
# 创建Timestamp时，可以传入时区信息：
stamp_moscow = pd.Timestamp('2011-03-12 04:00', tz='Europe/Moscow')
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

时区意识型Timestamp对象在内部保存了一个UTC时间戳值（自UNIX纪元1970年1月1日算起的纳秒数）。这个UTC值在时区转换过程中是不会发生变化的：

In [123]:
stamp_utc.value

1299902400000000000L

In [124]:
stamp_utc.tz_convert('US/Eastern').value

1299902400000000000L

In [125]:
# 使用pandas的DateOffset对象执行时间算术运算时，运算过程会自动关注是否存在夏令时转变期：

#夏令时转变前30分钟
from pandas.tseries.offsets import Hour

stamp = pd.Timestamp('2012-03-12 01:30', tz='US/Eastern')
stamp

Timestamp('2012-03-12 01:30:00-0400', tz='US/Eastern')

In [126]:
stamp + Hour()

Timestamp('2012-03-12 02:30:00-0400', tz='US/Eastern')

In [127]:
stamp = pd.Timestamp('2012-11-04 00:30', tz='US/Eastern')
stamp

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

In [128]:
stamp + 2 * Hour()

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

### 不同时区之间的运算

## 时期及其算术运算